In [ ]:
# 3
from keras.layers import Input, Dense, Flatten, Concatenate, Conv2D, MaxPooling2D, Dropout, LSTM, RepeatVector
from keras.optimizers import RMSprop
from keras.models import Model

def generate_model():
    word_input = Input(shape=(PADDING_SIZE, N_ONEHOT_WORD))
    image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))
    
    img = Conv2D(32, (3, 3), padding='same', activation='relu')(image_input)
    img = Conv2D(32, (3, 3), padding='same', activation='relu')(image_input)
    img = MaxPooling2D((4,4))(img)
    img = Conv2D(64, (3, 3), padding='same', activation='relu')(img)
    img = Conv2D(64, (3, 3), padding='same', activation='relu')(img)
    img = MaxPooling2D((4,4))(img)
    img = Dropout(0.25)(img)
    img = Conv2D(128, (3, 3), padding='same', activation='relu')(img)
    img = Conv2D(128, (3, 3), padding='same', activation='relu')(img)
    img = MaxPooling2D()(img)
    
    img = Flatten()(img)
    img = Dense(1024, activation='relu')(img)
    img = Dropout(0.3)(img)
    img = Dense(1024, activation='relu')(img)
    img = Dropout(0.3)(img)
    
    img = RepeatVector(PADDING_SIZE)(img)
    
    w = LSTM(64, return_sequences=True)(word_input)
    w = LSTM(64, return_sequences=True)(w)
    
    x = Concatenate()([w, img])
    x = LSTM(512, return_sequences=True, dropout=0.75)(x)
    x = LSTM(512)(x)
    
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    outputs = Dense(N_ONEHOT_WORD, activation='softmax')(x)

    optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
    model = Model(inputs=[word_input, image_input], outputs=outputs)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

print(generate_model().summary())